# Mixed-Precision Experiments (Advanced)

## Objective

Study error accumulation in different precisions:
- float32 vs float64 error growth
- Mixed-precision iterative refinement
- Precision-dependent stability

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append("..")

from utils.floating_point_tools import naive_sum, kahan_sum
from utils.linear_algebra_utils import gaussian_elimination_with_pivoting

np.random.seed(42)
plt.rcParams["figure.figsize"] = (12, 6)

## Precision Comparison

| Type | Bits | Mantissa | Exponent | ε_mach |
|------|------|----------|----------|--------|
| float32 | 32 | 23 | 8 | 2^-23 ≈ 1.2e-7 |
| float64 | 64 | 52 | 11 | 2^-52 ≈ 2.2e-16 |

In [ ]:
# Compare summation in different precisions
sizes = np.logspace(2, 6, 15).astype(int)
errors_32 = []
errors_64 = []

for n in sizes:
    arr = np.random.randn(n)
    exact = np.sum(arr.astype(np.float128))
    
    sum_32 = np.sum(arr.astype(np.float32))
    sum_64 = np.sum(arr.astype(np.float64))
    
    errors_32.append(abs(sum_32 - exact) / abs(exact))
    errors_64.append(abs(sum_64 - exact) / abs(exact))

plt.figure(figsize=(10, 6))
plt.loglog(sizes, errors_32, "o-", label="float32", linewidth=2)
plt.loglog(sizes, errors_64, "s-", label="float64", linewidth=2)
plt.loglog(sizes, sizes * np.finfo(np.float32).eps, "--", label=r"$n \varepsilon_{32}$", alpha=0.5)
plt.loglog(sizes, sizes * np.finfo(np.float64).eps, "--", label=r"$n \varepsilon_{64}$", alpha=0.5)
plt.xlabel("Array size")
plt.ylabel("Relative error")
plt.title("Summation Error: float32 vs float64")
plt.legend()
plt.grid(True, alpha=0.3)
plt.savefig("../plots/07_mixed_precision_summation.png", dpi=150, bbox_inches="tight")
plt.show()

## Iterative Refinement

Solve Ax = b using mixed precision:
1. Compute approximate solution in low precision
2. Compute residual in high precision
3. Solve for correction
4. Refine solution

In [ ]:
# Mixed-precision iterative refinement
def iterative_refinement(A, b, max_iter=5):
    """Solve Ax=b with iterative refinement."""
    n = len(b)
    
    # Initial solve in float32
    A_32 = A.astype(np.float32)
    b_32 = b.astype(np.float32)
    x = np.linalg.solve(A_32, b_32).astype(np.float64)
    
    errors = []
    
    for i in range(max_iter):
        # Compute residual in float64
        r = b - A @ x
        
        # Solve for correction in float32
        delta = np.linalg.solve(A_32, r.astype(np.float32)).astype(np.float64)
        
        # Update solution
        x = x + delta
        
        # Track error
        errors.append(np.linalg.norm(r))
    
    return x, errors

# Test
n = 50
A = np.random.randn(n, n)
x_true = np.random.randn(n)
b = A @ x_true

x_refined, residuals = iterative_refinement(A, b)

plt.figure(figsize=(10, 6))
plt.semilogy(residuals, "o-", linewidth=2)
plt.xlabel("Iteration")
plt.ylabel("Residual norm")
plt.title("Iterative Refinement Convergence")
plt.grid(True, alpha=0.3)
plt.savefig("../plots/07_iterative_refinement.png", dpi=150, bbox_inches="tight")
plt.show()

print(f"Final error: {np.linalg.norm(x_refined - x_true) / np.linalg.norm(x_true):.2e}")

## Key Takeaways

1. **float32 has ~7 decimal digits**, float64 has ~16
2. **Error scales with precision**: ε_32 ≈ 10^9 × ε_64
3. **Mixed precision** can improve efficiency while maintaining accuracy
4. **Iterative refinement** recovers full precision from low-precision solve